In [2]:
from src.get_data import load_benchmark
from src.normalization import get_adj_feats
from src.args import get_args
from src.models import get_model
from src.utils import accuracy
import torch.optim as optim
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import random

ModuleNotFoundError: No module named 'cpnet'

In [2]:
# load dataset


# all tensor, dense
dataset_name = 'cora'
# dataset_name = input('input dataset name: cora/citeseer/pubmed/...')


adj, feats, labels, idx_train, idx_val, idx_test = load_benchmark(dataset_name)

Loading cora dataset...
finish load data


In [3]:
# get args

# model_name = input('choose model: GCN/SGC/GFNN/GFN/AGNN/GIN/...')
model_name = 'GCN'
args = get_args(model_opt = model_name, dataset = dataset_name)

In [4]:
# get input for model

adj, feats = get_adj_feats(adj = adj, feats = feats, model_opt = model_name, degree = args.degree)

for GCN, return sym_norm(A) and raw feats


In [5]:
# processing

nb_class = (torch.max(labels) + 1).numpy()
Y_onehot =  torch.zeros(labels.shape[0], nb_class).scatter_(1, labels.unsqueeze(-1), 1)

nb_each_class_train = torch.sum(Y_onehot[idx_train], dim = 0)
nb_each_class_inv_train = torch.tensor(np.power(nb_each_class_train.numpy(), -1).flatten())
nb_each_class_inv_mat_train = torch.diag(nb_each_class_inv_train)

nb_each_class_val = torch.sum(Y_onehot[idx_val], dim = 0)
nb_each_class_inv_val = torch.tensor(np.power(nb_each_class_val.numpy(), -1).flatten())
nb_each_class_inv_mat_val = torch.diag(nb_each_class_inv_val)

nb_each_class_test = torch.sum(Y_onehot[idx_test], dim = 0)
nb_each_class_inv_test = torch.tensor(np.power(nb_each_class_test.numpy(), -1).flatten())
nb_each_class_inv_mat_test = torch.diag(nb_each_class_inv_test)

In [5]:
# get model
model = get_model(model_opt = model_name, nfeat = feats.size(1), \
                  nclass = labels.max().item()+1, nhid = args.hidden, \
                  dropout = args.dropout, cuda = args.cuda, \
                  dataset = dataset_name, degree = args.degree)
# optimizer
optimizer = optim.Adam(model.parameters(),
                           lr=args.lr, weight_decay=args.weight_decay)

# if args.cuda:
#     if model_name!='AGNN' and model_name!='GIN':
#         model.cuda()
#         feats = feats.cuda()
#         adj = adj.cuda()
#         labels = labels.cuda()
#         idx_train = idx_train.cuda()
#         idx_val = idx_val.cuda()
#         idx_test = idx_test.cuda()
    
    
# Print model's state_dict    
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor,"\t",model.state_dict()[param_tensor].size()) 
print("optimizer's state_dict:")

# Print optimizer's state_dict
for var_name in optimizer.state_dict():
    print(var_name,"\t",optimizer.state_dict()[var_name])
    
# Print parameters
for name, param in model.named_parameters():
    print(name, param)

Model's state_dict:
gc1.weight 	 torch.Size([1433, 16])
gc1.bias 	 torch.Size([16])
gc2.weight 	 torch.Size([16, 7])
gc2.bias 	 torch.Size([7])
optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0005, 'amsgrad': False, 'params': [0, 1, 2, 3]}]
gc1.weight Parameter containing:
tensor([[ 0.2102, -0.2396,  0.1495,  ...,  0.0294,  0.1798, -0.0105],
        [-0.1021,  0.1460,  0.0942,  ...,  0.0229,  0.0329,  0.1903],
        [ 0.0431, -0.0676,  0.0863,  ..., -0.0159, -0.1731,  0.1386],
        ...,
        [ 0.1948,  0.0915, -0.2434,  ..., -0.0467, -0.2098,  0.1553],
        [-0.0135, -0.2178,  0.0473,  ...,  0.2128, -0.0957,  0.0534],
        [-0.1084, -0.0559, -0.1729,  ...,  0.1303,  0.1701, -0.0910]],
       requires_grad=True)
gc1.bias Parameter containing:
tensor([-0.1308,  0.1932,  0.1251,  0.1600, -0.0565,  0.1765, -0.0049, -0.1707,
         0.0613,  0.1943, -0.1136,  0.0755, -0.2350,  0.1963, -0.2077, -0.1340],
  

In [6]:
delt = 0.05

In [7]:
# train, test


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output, fp1, fp2 = model(feats, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    model.eval()
    output, fp1, fp2 = model(feats, adj)
    
    CE_loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    loss_val = CE_loss_val
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    CE_loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    loss_test = CE_loss_test
    acc_test = accuracy(output[idx_test], labels[idx_test])
    
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'loss_test: {:.4f}'.format(loss_test.item()),
          'acc_test: {:.4f}'.format(acc_test.item()),
          'time: {:.4f}s'.format(time.time() - t))
    print("-------------------------------------------------")

    return epoch+1, loss_train.item(), acc_train.item(), loss_val.item(), \
            acc_val.item(), loss_test.item(), acc_test.item(), time.time() - t, \
            output, fp1, fp2
            




In [10]:
training_log = []

# Train model
t_total = time.time()
temp_val_loss = 999999
temp_val_acc = 0
temp_test_loss = 0
temp_test_acc = 0

for epoch in range(args.epochs):


    epo, trainloss, trainacc, valloss, valacc, testloss, testacc, epotime, output, fp1, fp2 = train(epoch)
    training_log.append([epo, trainloss, trainacc, valloss, valacc, testloss, testacc, epotime])

    if valacc >= temp_val_acc:
        temp_val_loss = valloss
        temp_val_acc = valacc
        temp_test_loss = testloss
        temp_test_acc = testacc
        propagation_feats = fp1
        before_softmax = fp2
        after_softmax = output
        
        if model_name == 'AFGNN':
            temp_weight = torch.softmax(model.state_dict()['gc1.linear_weight'].data,dim=0)




Epoch: 0001 loss_train: 0.1928 acc_train: 1.0000 loss_val: 0.7616 acc_val: 0.7960 loss_test: 0.7308 acc_test: 0.8160 time: 0.0330s
-------------------------------------------------
Epoch: 0002 loss_train: 0.1921 acc_train: 1.0000 loss_val: 0.7620 acc_val: 0.7940 loss_test: 0.7310 acc_test: 0.8150 time: 0.0427s
-------------------------------------------------
Epoch: 0003 loss_train: 0.1913 acc_train: 1.0000 loss_val: 0.7613 acc_val: 0.7940 loss_test: 0.7305 acc_test: 0.8130 time: 0.0231s
-------------------------------------------------
Epoch: 0004 loss_train: 0.1905 acc_train: 1.0000 loss_val: 0.7610 acc_val: 0.7940 loss_test: 0.7302 acc_test: 0.8130 time: 0.0275s
-------------------------------------------------
Epoch: 0005 loss_train: 0.1897 acc_train: 1.0000 loss_val: 0.7598 acc_val: 0.7940 loss_test: 0.7291 acc_test: 0.8150 time: 0.0197s
-------------------------------------------------
Epoch: 0006 loss_train: 0.1890 acc_train: 1.0000 loss_val: 0.7594 acc_val: 0.7940 loss_test: 0.

Epoch: 0053 loss_train: 0.1630 acc_train: 1.0000 loss_val: 0.7387 acc_val: 0.7960 loss_test: 0.7103 acc_test: 0.8110 time: 0.0179s
-------------------------------------------------
Epoch: 0054 loss_train: 0.1626 acc_train: 1.0000 loss_val: 0.7411 acc_val: 0.7920 loss_test: 0.7119 acc_test: 0.8130 time: 0.0206s
-------------------------------------------------
Epoch: 0055 loss_train: 0.1622 acc_train: 1.0000 loss_val: 0.7375 acc_val: 0.7960 loss_test: 0.7094 acc_test: 0.8120 time: 0.0162s
-------------------------------------------------
Epoch: 0056 loss_train: 0.1618 acc_train: 1.0000 loss_val: 0.7409 acc_val: 0.7920 loss_test: 0.7117 acc_test: 0.8120 time: 0.0190s
-------------------------------------------------
Epoch: 0057 loss_train: 0.1614 acc_train: 1.0000 loss_val: 0.7378 acc_val: 0.7960 loss_test: 0.7095 acc_test: 0.8120 time: 0.0164s
-------------------------------------------------
Epoch: 0058 loss_train: 0.1610 acc_train: 1.0000 loss_val: 0.7388 acc_val: 0.7920 loss_test: 0.

Epoch: 0107 loss_train: 0.1461 acc_train: 1.0000 loss_val: 0.7287 acc_val: 0.7900 loss_test: 0.7009 acc_test: 0.8120 time: 0.0175s
-------------------------------------------------
Epoch: 0108 loss_train: 0.1459 acc_train: 1.0000 loss_val: 0.7270 acc_val: 0.7920 loss_test: 0.6996 acc_test: 0.8130 time: 0.0201s
-------------------------------------------------
Epoch: 0109 loss_train: 0.1456 acc_train: 1.0000 loss_val: 0.7283 acc_val: 0.7920 loss_test: 0.7005 acc_test: 0.8110 time: 0.0170s
-------------------------------------------------
Epoch: 0110 loss_train: 0.1454 acc_train: 1.0000 loss_val: 0.7267 acc_val: 0.7920 loss_test: 0.6994 acc_test: 0.8130 time: 0.0182s
-------------------------------------------------
Epoch: 0111 loss_train: 0.1451 acc_train: 1.0000 loss_val: 0.7283 acc_val: 0.7920 loss_test: 0.7005 acc_test: 0.8110 time: 0.0267s
-------------------------------------------------
Epoch: 0112 loss_train: 0.1449 acc_train: 1.0000 loss_val: 0.7271 acc_val: 0.7920 loss_test: 0.

Epoch: 0161 loss_train: 0.1359 acc_train: 1.0000 loss_val: 0.7219 acc_val: 0.7920 loss_test: 0.6947 acc_test: 0.8120 time: 0.0186s
-------------------------------------------------
Epoch: 0162 loss_train: 0.1358 acc_train: 1.0000 loss_val: 0.7201 acc_val: 0.7920 loss_test: 0.6934 acc_test: 0.8150 time: 0.0182s
-------------------------------------------------
Epoch: 0163 loss_train: 0.1356 acc_train: 1.0000 loss_val: 0.7213 acc_val: 0.7920 loss_test: 0.6942 acc_test: 0.8120 time: 0.0179s
-------------------------------------------------
Epoch: 0164 loss_train: 0.1355 acc_train: 1.0000 loss_val: 0.7197 acc_val: 0.7920 loss_test: 0.6930 acc_test: 0.8150 time: 0.0171s
-------------------------------------------------
Epoch: 0165 loss_train: 0.1353 acc_train: 1.0000 loss_val: 0.7204 acc_val: 0.7920 loss_test: 0.6936 acc_test: 0.8150 time: 0.0183s
-------------------------------------------------
Epoch: 0166 loss_train: 0.1352 acc_train: 1.0000 loss_val: 0.7209 acc_val: 0.7920 loss_test: 0.

In [9]:
# print result



print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Best result:",
          "val_loss=",temp_val_loss,
            "test_loss=",temp_test_loss,
             "test_acc=",temp_test_acc)


Optimization Finished!
Total time elapsed: 4.1452s
Best result: val_loss= 0.8655314445495605 test_loss= 0.8287785053253174 test_acc= 0.816
